In [1]:
import eons
import neuro
import speech2spikes
import os
import torchaudio
import torch

PATH_SOUNDS = '/home/joao/dev/MLAudio/shotspotter/data/gunshots'

# Look into possible enabling cumsum (see github)
s2s = speech2spikes.S2S()

# workaround to configure to 12khz sample rate (s2s.configure() didnt work)
s2s._default_spec_kwargs = {
    "sample_rate": 12000,
    "n_mels": 20,
    "n_fft": 512,
    "f_min": 20,
    "f_max": 4000,
    "hop_length": 80,
}
s2s.transform = torchaudio.transforms.MelSpectrogram(**s2s._default_spec_kwargs)

# Some constants
NUM_INPUT_NEURONS = 80 # see paper
NUM_OUTPUT_NEURONS = 2

In [2]:
file_paths = [PATH_SOUNDS+'/'+fn for fn in os.listdir(PATH_SOUNDS)]

In [33]:
# we want to convert our sound file into spikes
def to_spikes(path):
    data, rate = torchaudio.load(path)

    trains, targets = s2s([(data, torch.tensor(0))])
    print(trains.shape)

    trains = trains[0]
    
    print(trains[0][0])
    # LOOK INTO EXACTLY WHAT CUMSUM IS DOING so we can figure out conversion to spikes
    # with cumsum=False (default) looks like it makes sense
    # I think we dont want cumsum (not entirely sure of what its doing)

    spikes = []
    for i in range(2):
        for channel in trains[i]:
            pos_channel, neg_channel = [], []
            for t in channel:
                if t == 1:
                    pos_channel.append(1)
                    neg_channel.append(0)
                elif t == -1:
                    neg_channel.append(1)
                    pos_channel.append(0)
                else:
                    pos_channel.append(0)
                    neg_channel.append(0)

            spikes.append(pos_channel.copy())
            spikes.append(neg_channel.copy())

    return spikes
            
spikes = to_spikes(file_paths[0])

torch.Size([1, 2, 20, 301])
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
         0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -1., -1.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  1.,  0.

In [ ]:
dir(neuro.Spike)

In [37]:
print(spikes[0])
print(spikes[1])

80